In [4]:
pip install cplex

   ---------------------------------------- 0.0/26.3 MB ? eta -:--:--
    --------------------------------------- 0.5/26.3 MB 2.8 MB/s eta 0:00:10
    --------------------------------------- 0.5/26.3 MB 2.8 MB/s eta 0:00:10
   - -------------------------------------- 0.8/26.3 MB 1.0 MB/s eta 0:00:25
   - -------------------------------------- 1.3/26.3 MB 1.7 MB/s eta 0:00:15
   -- ------------------------------------- 1.8/26.3 MB 1.7 MB/s eta 0:00:15
   ---- ----------------------------------- 2.9/26.3 MB 2.3 MB/s eta 0:00:11
   ------ --------------------------------- 4.2/26.3 MB 3.0 MB/s eta 0:00:08
   ------- -------------------------------- 5.0/26.3 MB 3.2 MB/s eta 0:00:07
   -------- ------------------------------- 5.5/26.3 MB 3.0 MB/s eta 0:00:07
   --------- ------------------------------ 6.6/26.3 MB 3.3 MB/s eta 0:00:06
   ----------- ---------------------------- 7.9/26.3 MB 3.5 MB/s eta 0:00:06
   ------------ --------------------------- 8.4/26.3 MB 3.6 MB/s eta 0:00:05
   ---

In [5]:
import cplex

In [8]:
model = cplex.Cplex()

In [62]:
# Création du modèle
model = cplex.Cplex()
model.objective.set_sense(model.objective.sense.minimize)

# Variables de production (x)
coef_obj = [2.8, 1.5, 4.3, 3.7]  
lb = [0, 0, 0, 0]  
ub = [70, 20, 60, 45]  
v_names = ["x1", "x2", "x3", "x4"]
 

model.variables.add(obj=coef_obj, lb=lb, ub=ub, names=v_names)

# Variables binaires pour activer les techniques (y)
y_names = ["y1", "y2", "y3", "y4"]

y_coef_obj = [100, 138, 76, 98]  

model.variables.add(obj=y_coef_obj, lb=[0] * 4, ub=[1] * 4, names=y_names)

# Contraintes
constraints = [
    [["x1", "x2", "x3", "x4"], [1, 1, 1, 1]],  
    [["y1", "y2", "y3", "y4"], [1, 1, 1, 1]]  
]
rhs = [100, 3]
c_sense = ["G", "L"]  # G >=, L <=

model.linear_constraints.add(lin_expr=constraints, senses=c_sense, rhs=rhs, names=["Production", "Techniques"])

# Contraintes de liaison entre x et y : x_t ≤ u_t * y_t
for i, (x, y, u) in enumerate(zip(v_names, y_names, ub)):
    model.linear_constraints.add(
        lin_expr=[[[x, y], [1, -u]]],
        senses=["L"],
        rhs=[0],
        names=[f"liaison_{i+1}"]
    )

# Résolution
model.solve()

# Vérification du statut avant d'extraire la solution
status = model.solution.get_status()
if status in [1, 101, 102]:  # 1: Optimal, 101: Feasible, 102: Feasible relaxed
    solution_x = model.solution.get_values(v_names)
    solution_y = model.solution.get_values(y_names)
    print("Solution x :", solution_x)
    print("Techniques utilisées (y) :", solution_y)
    print("Coût total :", model.solution.get_objective_value())
else:
    print("Problème infaisable ou erreur dans la modélisation. Vérifiez les contraintes.")

Version identifier: 22.1.2.0 | 2024-12-09 | 8bd2200c8
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
Aggregator did 4 substitutions.
Reduced LP has 2 rows, 4 columns, and 8 nonzeros.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =           463.000000
Solution x : [70.0, 0.0, 30.0, 0.0]
Techniques utilisées (y) : [1.0, 0.0, 0.5, 0.0]
Coût total : 463.0


In [50]:
# question 2
model.write("model.lp")

In [65]:
model.solution.write("solution.lp")